<h1>Assignment</h1>

In [3]:
# Importing libraries

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

<h2>Part A - Calculating loyalty points</h2>


Import Deposit Data

In [4]:
# Converting the data into dataframe

data2 = pd.read_csv('Deposit Data - Sheet1.csv')
deposit_df = pd.DataFrame(data2)

In [5]:
deposit_df.head()

,User_Id,Datetime,Amount
0,357,01-10-2022 0:03,2000
1,776,01-10-2022 0:03,2500
2,492,01-10-2022 0:06,5000
3,803,01-10-2022 0:07,5000
4,875,01-10-2022 0:09,1500


In [18]:
# Q1 Find Playerwise Loyalty points earned by Players in the following slots:-
#     a. 2nd October Slot S1
#     b. 16th October Slot S2
#     b. 18th October Slot S1
#     b. 26th October Slot S2

# Reading the deposit and withdrawal data
deposit_data = pd.read_csv("Deposit Data - Sheet1.csv")
withdrawal_data = pd.read_csv("Withdrawl Data - Sheet1.csv")

# Preprocess the data
deposit_data['Datetime'] = pd.to_datetime(deposit_data['Datetime'], format='%d-%m-%Y %H:%M')
withdrawal_data['Datetime'] = pd.to_datetime(withdrawal_data['Datetime'], format='%d-%m-%Y %H:%M')

# Assign time slots (S1: 12am-12pm, S2: 12pm-12am)
def assign_slot(datetime):
    return "S1" if datetime.hour < 12 else "S2"

deposit_data['Slot'] = deposit_data['Datetime'].apply(assign_slot)
withdrawal_data['Slot'] = withdrawal_data['Datetime'].apply(assign_slot)

# Extract the date for easier filtering
deposit_data['Date'] = deposit_data['Datetime'].dt.date
withdrawal_data['Date'] = withdrawal_data['Datetime'].dt.date

# Function to calculate loyalty points for a specific date and slot
def calculate_slot_points(date, slot):
    # Filter deposit and withdrawal data for the given date and slot
    deposits = deposit_data[(deposit_data['Date'] == date) & (deposit_data['Slot'] == slot)]
    withdrawals = withdrawal_data[(withdrawal_data['Date'] == date) & (withdrawal_data['Slot'] == slot)]
    
    # Summarize deposits and withdrawals by user
    deposit_summary = deposits.groupby('User_Id')['Amount'].sum().reset_index(name='Deposit_Amount')
    withdrawal_summary = withdrawals.groupby('User_Id')['Amount'].sum().reset_index(name='Withdrawal_Amount')
    
    # Count the number of deposit and withdrawal transactions
    deposit_counts = deposits.groupby('User_Id').size().reset_index(name='Deposit_Count')
    withdrawal_counts = withdrawals.groupby('User_Id').size().reset_index(name='Withdrawal_Count')
    
    # Merge all summaries into a single dataset
    merged = pd.merge(deposit_summary, withdrawal_summary, on='User_Id', how='outer').fillna(0)
    merged = pd.merge(merged, deposit_counts, on='User_Id', how='outer').fillna(0)
    merged = pd.merge(merged, withdrawal_counts, on='User_Id', how='outer').fillna(0)
    
    # Calculate loyalty points based on the given formula
    merged['Loyalty_Points'] = (
        0.01 * merged['Deposit_Amount'] +
        0.005 * merged['Withdrawal_Amount'] +
        0.001 * merged[['Deposit_Count', 'Withdrawal_Count']].apply(lambda x: max(x[0] - x[1], 0), axis=1)
    )
    
    # Return the results sorted by loyalty points in descending order
    return merged[['User_Id', 'Loyalty_Points']].sort_values(by='Loyalty_Points', ascending=False)

# Define the specific dates and slots we want to analyze
dates_slots = [
    ("2022-10-02", "S1"),
    ("2022-10-16", "S2"),
    ("2022-10-18", "S1"),
    ("2022-10-26", "S2")
]

# Calculate and display loyalty points for each date and slot
for date, slot in dates_slots:
    result = calculate_slot_points(pd.to_datetime(date).date(), slot)
    print(f"\nLoyalty Points for {date} Slot {slot}:\n", result.head())



Loyalty Points for 2022-10-02 Slot S1:
      User_Id  Loyalty_Points
150      634        1478.355
158      672        1300.000
133      566        1250.206
234      949         677.500
104      446         550.002

Loyalty Points for 2022-10-16 Slot S2:
      User_Id  Loyalty_Points
135      634        1491.555
41       212         999.991
17        99         980.002
6         28         900.004
121      566         880.003

Loyalty Points for 2022-10-18 Slot S1:
      User_Id  Loyalty_Points
134      634        2723.100
37       208        1700.001
147      673         900.001
31       162         770.000
46       245         750.000

Loyalty Points for 2022-10-26 Slot S2:
      User_Id  Loyalty_Points
156      714        2000.001
85       369        1501.715
141      634        1236.810
118      538        1200.003
0          2         900.002


In [19]:
# Q2 Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October. 
#      In case of tie, number of games played should be taken as the next criteria for ranking.

# Load the datasets
deposit_df = pd.read_csv("Deposit Data - Sheet1.csv")
withdrawal_df = pd.read_csv("Withdrawl Data - Sheet1.csv")

# Convert 'Datetime' columns to datetime objects
deposit_df['Datetime'] = pd.to_datetime(deposit_df['Datetime'], format='%d-%m-%Y %H:%M')
withdrawal_df['Datetime'] = pd.to_datetime(withdrawal_df['Datetime'], format='%d-%m-%Y %H:%M')

# Extract the month from 'Datetime'
deposit_df['Month'] = deposit_df['Datetime'].dt.month
withdrawal_df['Month'] = withdrawal_df['Datetime'].dt.month

# Filter data for October
oct_deposits = deposit_df[deposit_df['Month'] == 10]
oct_withdrawals = withdrawal_df[withdrawal_df['Month'] == 10]

# Aggregate data by User_Id
oct_deposit_summary = oct_deposits.groupby('User_Id')['Amount'].sum().reset_index(name='Deposit_Amount')
oct_withdrawal_summary = oct_withdrawals.groupby('User_Id')['Amount'].sum().reset_index(name='Withdrawal_Amount')

oct_deposit_counts = oct_deposits.groupby('User_Id').size().reset_index(name='Deposit_Count')
oct_withdrawal_counts = oct_withdrawals.groupby('User_Id').size().reset_index(name='Withdrawal_Count')

# Merge aggregated data into a single DataFrame
oct_merged = pd.merge(oct_deposit_summary, oct_withdrawal_summary, on='User_Id', how='outer').fillna(0)
oct_merged = pd.merge(oct_merged, oct_deposit_counts, on='User_Id', how='outer').fillna(0)
oct_merged = pd.merge(oct_merged, oct_withdrawal_counts, on='User_Id', how='outer').fillna(0)

# Add a placeholder for games played (replace with actual data if available)
oct_merged['Games_Played'] = 50  # Replace with real data for 'Games_Played' if available

# Calculate loyalty points
oct_merged['Loyalty_Points'] = (
    0.01 * oct_merged['Deposit_Amount'] +
    0.005 * oct_merged['Withdrawal_Amount'] +
    0.001 * oct_merged[['Deposit_Count', 'Withdrawal_Count']].apply(lambda x: max(x[0] - x[1], 0), axis=1) +
    0.2 * oct_merged['Games_Played']
)

# Rank players by loyalty points (and by games played in case of ties)
oct_merged['Rank'] = oct_merged.sort_values(
    by=['Loyalty_Points', 'Games_Played'], ascending=[False, False]
).rank(method='first', ascending=True)['Loyalty_Points']

# Sort by rank to get the final leaderboard
oct_ranked = oct_merged.sort_values(by='Rank')

# Display the top 10 players
print("\nTop 10 Players for October:\n", oct_ranked[['User_Id', 'Loyalty_Points', 'Games_Played', 'Rank']].head(10))

# Save the full ranked list to a CSV file if needed
oct_ranked.to_csv("October_Player_Rankings.csv", index=False)



Top 10 Players for October:
      User_Id  Loyalty_Points  Games_Played  Rank
361      387          12.501            50   1.0
90        98          13.001            50   2.0
174      188          13.001            50   3.0
158      172          14.004            50   4.0
84        92          15.001            50   5.0
518      552          15.001            50   6.0
575      617          15.001            50   7.0
795      847          18.003            50   8.0
790      842          19.007            50   9.0
57        65          20.001            50  10.0


In [6]:
# Q3:-The average deposit amount

Sum_of_amount = deposit_df.groupby('User_Id')['Amount'].sum()
Average_of_amount = Sum_of_amount.mean()
print("The Average Deposit Amount is:", Average_of_amount)

The Average Deposit Amount is: 104669.64918032786


In [7]:
# Q4:- The average deposit amount per user in a month

# Convert Datetime column to Datetime type
deposit_df['Datetime'] = pd.to_datetime(deposit_df['Datetime'], format='%d-%m-%Y %H:%M')

# Extract year, month from the datetime
deposit_df['month_year'] = deposit_df['Datetime'].dt.to_period('M')

# Group by user_id and month-year, and sum the amount
Sum_of_amount_per_month = deposit_df.groupby(['User_Id', 'month_year'])['Amount'].sum().reset_index()

# Finally find the average
Average_of_per_month = Sum_of_amount_per_month.groupby(['User_Id','month_year']).mean()
print("The average deposit amount per user in a month is:", Average_of_per_month)

The average deposit amount per user in a month is:                       Amount
User_Id month_year          
1       2022-10       5000.0
2       2022-10     567000.0
3       2022-10      40000.0
4       2022-10       1750.0
5       2022-10      74100.0
...                      ...
994     2022-10       1400.0
995     2022-10      34500.0
996     2022-10      96400.0
998     2022-10       7400.0
999     2022-10       9000.0

[915 rows x 1 columns]


Import User Gameplay Data

In [8]:
# Converting the data into dataframe

data2 = pd.read_csv('User Gameplay Data - Sheet1.csv')
Gameplay_df = pd.DataFrame(data2)

In [9]:
Gameplay_df.head(50)

,User_ID,Games_Played,Datetime
0,851,12-31-1899 0:00,01-10-2022 0:00
1,717,12-31-1899 0:00,01-10-2022 0:00
2,456,12-31-1899 0:00,01-10-2022 0:00
3,424,12-31-1899 0:00,01-10-2022 0:00
4,845,12-31-1899 0:00,01-10-2022 0:00
5,15,12-31-1899 0:00,01-10-2022 0:00
6,738,12-31-1899 0:00,01-10-2022 0:00
7,748,12-31-1899 0:00,01-10-2022 0:01
8,582,12-31-1899 0:00,01-10-2022 0:01
9,39,12-31-1899 0:00,01-10-2022 0:01


In [10]:
# Q5:- The average number of games played by per user

Gameplay_df['Games_Played'] = pd.to_numeric(Gameplay_df['Games_Played'],errors = 'coerce')
Gameplay_df['Games_Played'] = Gameplay_df['Games_Played'].fillna(0)

Count_Games = Gameplay_df.groupby('User_ID')['Games_Played'].sum()
Average_of_GamesPlayed = Count_Games.mean()

print('Games played by per user:',Count_Games)
print('\nThe average number of games played by per user:',Average_of_GamesPlayed)

Games played by per user: User_ID
0       15.0
1        7.0
2       95.0
3       80.0
4        5.0
       ...  
995     16.0
996    467.0
997    107.0
998      4.0
999    171.0
Name: Games_Played, Length: 1000, dtype: float64

The average number of games played by per user: 348.403


<h2>Part B - How much bonus should be allocated to leaderboard players?</h2>

In [20]:
# Q1 Only top 50 ranked players are awarded bonus

# Calculate total loyalty points and rank players for October
oct_merged['Rank'] = oct_merged['Loyalty_Points'].rank(ascending=False, method='first')
top_50_players = oct_merged[oct_merged['Rank'] <= 50].copy()

# Total loyalty points of the top 50 players
total_loyalty_points_top_50 = top_50_players['Loyalty_Points'].sum()

# Allocate bonuses proportionally
total_bonus_pool = 50000  # ₹50,000
top_50_players['Bonus'] = (top_50_players['Loyalty_Points'] / total_loyalty_points_top_50) * total_bonus_pool

# Final leaderboard with bonuses
top_50_players = top_50_players.sort_values(by='Rank')
print("\nTop 50 Players with Bonus Allocation:\n", top_50_players[['User_Id', 'Loyalty_Points', 'Rank', 'Bonus']])



Top 50 Players with Bonus Allocation:
      User_Id  Loyalty_Points  Rank        Bonus
592      634       83848.525   1.0  6689.953097
91        99       23673.737   2.0  1888.836925
628      672       22765.780   3.0  1816.394509
194      212       22209.082   4.0  1771.977705
693      740       19221.424   5.0  1533.603901
532      566       19127.155   6.0  1526.082538
668      714       16773.034   7.0  1338.256228
393      421       15145.060   8.0  1208.366409
344      369       14441.044   9.0  1152.195665
28        30       14060.775  10.0  1121.855456
550      587       13502.081  11.0  1077.279399
204      222       13356.803  12.0  1065.688223
328      352       12988.050  13.0  1036.266831
149      162       12493.200  14.0   996.784643
861      920       12359.400  15.0   986.109252
387      415       12309.215  16.0   982.105183
535      569       12287.823  17.0   980.398397
340      365       12131.675  18.0   967.939946
736      786       12105.816  19.0   965.876755


<h2>Part C</h2>

Q. Should they base it on the amount of loyalty points? Should it be based on number of games? Or something else?

The bonus distribution should primarily be based on loyalty points. This method ensures fairness by rewarding players who are most engaged across multiple dimensions (deposits, withdrawals, and games played).

Q. Would you say the loyalty point formula is fair or unfair?

While the current loyalty point formula is solid, there’s room to improve it by making small tweaks that would better reflect the different ways players engage with the platform. By increasing the weight for withdrawals, adjusting the games played multiplier, and introducing new incentives for referrals and long-term commitment, the formula could be made more robust, fair, and better aligned with the company’s long-term goals of player retention and engagement.

Q. Can you suggest any way to make the loyalty point formula more robust?

The formula is pretty good, but I think by giving more weight to withdrawals, adjusting the games played part, and adding new rewards for things like referrals and long-term activity, it could be a bit more balanced. These changes would probably help encourage a better overall engagement and make players feel like their commitment to the platform is truly valued.

